In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Atuona,PF,2022-05-17 20:14:47,-9.8000,-139.0333,79.07,68,79,16.15
1,1,Arraial Do Cabo,BR,2022-05-17 20:14:47,-22.9661,-42.0278,73.26,68,20,10.36
2,2,Vilhena,BR,2022-05-17 20:14:48,-12.7406,-60.1458,76.30,45,59,5.93
3,3,Fairbanks,US,2022-05-17 20:11:37,64.8378,-147.7164,52.25,61,40,8.05
4,4,Bluff,NZ,2022-05-17 20:14:48,-46.6000,168.3333,41.81,81,80,11.01


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Atuona,PF,2022-05-17 20:14:47,-9.8000,-139.0333,79.07,68,79,16.15
2,2,Vilhena,BR,2022-05-17 20:14:48,-12.7406,-60.1458,76.30,45,59,5.93
12,12,Rikitea,PF,2022-05-17 20:14:52,-23.1203,-134.9692,76.19,79,13,21.45
16,16,Vaini,TO,2022-05-17 20:14:53,-21.2000,-175.2000,75.36,78,40,6.91
25,25,La Orilla,MX,2022-05-17 20:14:57,17.9833,-102.2333,88.27,40,100,12.75
32,32,Kapaa,US,2022-05-17 20:14:59,22.0752,-159.3190,80.58,92,75,12.66
34,34,Hilo,US,2022-05-17 20:15:00,19.7297,-155.0900,79.61,95,100,10.36
37,37,Kavaratti,IN,2022-05-17 20:15:01,10.5669,72.6420,82.09,85,100,20.22
43,43,Kavieng,PG,2022-05-17 20:15:03,-2.5744,150.7967,81.50,79,9,8.72
49,49,Puerto Baquerizo Moreno,EC,2022-05-17 20:16:06,-0.9000,-89.6000,81.59,65,40,11.50


In [13]:
preferred_cities_df.count()

City_ID       152
City          152
Country       152
Date          152
Lat           152
Lng           152
Max Temp      152
Humidity      152
Cloudiness    152
Wind Speed    152
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Atuona,PF,79.07,-9.8000,-139.0333,
2,Vilhena,BR,76.30,-12.7406,-60.1458,
12,Rikitea,PF,76.19,-23.1203,-134.9692,
16,Vaini,TO,75.36,-21.2000,-175.2000,
25,La Orilla,MX,88.27,17.9833,-102.2333,
32,Kapaa,US,80.58,22.0752,-159.3190,
34,Hilo,US,79.61,19.7297,-155.0900,
37,Kavaratti,IN,82.09,10.5669,72.6420,
43,Kavieng,PG,81.50,-2.5744,150.7967,
49,Puerto Baquerizo Moreno,EC,81.59,-0.9000,-89.6000,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Atuona,PF,79.07,-9.8000,-139.0333,Villa Enata
2,Vilhena,BR,76.30,-12.7406,-60.1458,Hotel Mirage
12,Rikitea,PF,76.19,-23.1203,-134.9692,People ThankYou
16,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
25,La Orilla,MX,88.27,17.9833,-102.2333,Baymont by Wyndham Lazaro Cardenas
32,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
34,Hilo,US,79.61,19.7297,-155.0900,Hilo Hawaiian Hotel
37,Kavaratti,IN,82.09,10.5669,72.6420,Hotels in Lakshadweep Islands
43,Kavieng,PG,81.50,-2.5744,150.7967,Nusa Island Retreat
49,Puerto Baquerizo Moreno,EC,81.59,-0.9000,-89.6000,Hotel los Algarrobos


In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [22]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))